## Ejercicio 1

In [ ]:
def readFile (filename): 
    '''Arguments: 
    filename – name of the spam dataset file 
    12 columns: 11 features/dimensions (X) + 1 column with labels (Y) 
    Y -- Train labels (0 if normal traffic, 1 if botnet)  
    m rows: number of examples (m) 
    Returns: 
    An RDD containing the data of filename. Each example (row) of the file 
    corresponds to one RDD record. Each record of the RDD is a tuple (X,y).  
    “X” is an array containing the 11 features (float number) of an example  
    “y” is the 12th column of an example (integer 0/1) '''
textFile, map

In [ ]:
def normalize (RDD_Xy): 
    '''Arguments: 
    RDD_Xy is an RDD containing data examples. Each record of the RDD is a tuple 
    (X,y). 
    “X” is an array containing the 11 features (float number) of an example 
    “y” is the label of the example (integer 0/1)  
    Returns: 
    An RDD rescaled to N(0,1) in each column (mean=0, standard deviation=1) '''
    # Para evitar que una variable tenga mucho mas peso que el resto, un map para la fila

In [ ]:
def train (RDD_Xy, iterations, learning_rate, lambda_reg): 
    '''Arguments: 
    RDD_Xy --- RDD containing data examples. Each record of the RDD is a tuple 
    (X,y). 
    “X” is an array containing the 11 features (float number) of an example 
    “y” is the label of the example (integer 0/1)  
    iterations -- number of iterations of the optimization loop 
    learning_rate -- learning rate of the gradient descent 
    lambda_reg – regularization rate: l2 es el que vamos a aplicar
    
    Returns: 
    A list or array containing the weights “w” and bias “b”	at the end of the 
    training process'''	

    # Si esta normalizado con 10 iteraciones va bien, en cada vuelta hay que hacer un print del coste 

In [ ]:
def accuracy (w, b, RDD_Xy): 
    '''Arguments: 
    w -- weights 
    b -- bias 
    RDD_Xy – RDD containing examples to be predicted  
    Returns: 
    accuracy -- the number of predictions that are correct divided by the number         
    of records (examples) in RDD_xy.  
    Predict function can be used for predicting a single example'''

In [ ]:
def predict (w, b, X): 
    '''Arguments: 
    w -- weights 
    b -- bias 
    X – Example to be predicted  
     
    Returns: 
    Y_pred – a value (0/1) corresponding to the prediction of X '''

In [ ]:
# read data
data=readFile(path)

# standarize
data = normalize(data)

ws = train(data,nIter,learningRate)
acc = accuracy(data,ws)
print("acc:",acc)

## Ejercicio 2

In [ ]:
# read data
data=readFile(path)

# standarize
data = normalize(data)
num_blocks_cv=10

# shuffle rows and transform data, specifying the number of blocks
data_cv = transform(data,num_blocks_cv)

for i in range(num_blocks_cv):
    tr_data,test_data=get_block_data(data_cv,i)
    
    ws = train(data,nIter,learningRate)
    acc = accuracy(data,ws)
    print("acc:",acc)

print("average acc:" avg_acc)